# Medical Prescription Model - Inference Only

This notebook loads the fine-tuned clinic chatbot model and provides an interface for generating medical prescriptions based on patient symptoms.


In [ ]:
import torch
import json
from unsloth import FastLanguageModel


## Load Fine-tuned Model

Loading the LoRA adapter merged with the base model.


In [ ]:
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="../models/clinic-chatbot-lora",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# Enable fast inference mode
FastLanguageModel.for_inference(model)

print("✅ Model loaded successfully!")


## Inference Function

Function to generate structured prescription responses.


In [ ]:
def get_prescription(patient_symptoms, max_new_tokens=512, temperature=0.3, top_p=0.9):
    """
    Generate prescription from patient symptoms.
    
    Args:
        patient_symptoms (str): Description of patient symptoms
        max_new_tokens (int): Maximum tokens to generate
        temperature (float): Sampling temperature (lower = more deterministic)
        top_p (float): Nucleus sampling parameter
    
    Returns:
        dict: Structured prescription with medicine details and speech
    """
    
    conversation = [
        {
            "role": "system",
            "content": "You are a professional medical doctor. When a patient describes their symptoms, provide a structured prescription response in JSON format with: prescription_text, medicine_name, dose_size, frequency, duration, and speech (natural language explanation)."
        },
        {
            "role": "user",
            "content": patient_symptoms
        }
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    
    try:
        prescription = json.loads(response)
        return prescription
    except json.JSONDecodeError:
        return {"raw_response": response, "error": "Failed to parse JSON response"}

print("✅ Inference function ready!")


## Test Cases

Run inference on multiple test cases.


In [ ]:
test_cases = [
    "I have leg pains and my nails are coming off",
]

print("🏥 Testing Fine-tuned Clinic Chatbot\n" + "="*80 + "\n")

for symptom in test_cases:
    print(f"PATIENT: {symptom}")
    prescription = get_prescription(symptom)
    print(f"DOCTOR PRESCRIPTION:")
    print(json.dumps(prescription, indent=2))
    
    if 'speech' in prescription:
        print(f"\n💬 DOCTOR SAYS: {prescription['speech']}")
    
    print("\n" + "-"*80 + "\n")


## Gradio Chat Interface

Interactive chat interface for the medical chatbot.


In [ ]:
import gradio as gr

def chat_interface(patient_symptoms, temperature, max_tokens):
    """
    Gradio interface wrapper for the prescription model.
    """
    if not patient_symptoms.strip():
        return "⚠️ Please describe your symptoms.", "{}"
    
    # Get prescription from model
    prescription = get_prescription(
        patient_symptoms, 
        max_new_tokens=int(max_tokens),
        temperature=float(temperature)
    )
    
    # Format the response
    if 'error' in prescription:
        return f"⚠️ Error: {prescription['error']}", json.dumps(prescription, indent=2)
    
    # Extract speech for display
    speech = prescription.get('speech', 'No speech generated')
    
    # Format prescription details nicely
    prescription_display = f"""
## 🏥 Doctor's Prescription

**Diagnosis:** {prescription.get('prescription_text', 'N/A')}

**Medicine:** {prescription.get('medicine_name', 'N/A')}

**Dosage:** {prescription.get('dose_size', 'N/A')}

**Frequency:** {prescription.get('frequency', 'N/A')}

**Duration:** {prescription.get('duration', 'N/A')}

---

### 💬 Doctor Says:
{speech}
"""
    
    # Return both the formatted display and JSON
    return prescription_display, json.dumps(prescription, indent=2)

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="🏥 Medical Chatbot") as demo:
    gr.Markdown("""
    # 🏥 Medical Prescription Chatbot
    
    ### Describe your symptoms and receive a medical prescription.
    
    ⚠️ **Disclaimer:** This is a fine-tuned AI model for educational purposes only. 
    Always consult a real healthcare professional for medical advice.
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            # Input area
            symptoms_input = gr.Textbox(
                label="🗣️ Patient Symptoms",
                placeholder="Describe your symptoms here (e.g., 'I have severe headache and nausea')",
                lines=3
            )
            
            # Advanced settings
            with gr.Accordion("⚙️ Advanced Settings", open=False):
                temperature_slider = gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.3,
                    step=0.1,
                    label="Temperature (lower = more consistent)",
                )
                max_tokens_slider = gr.Slider(
                    minimum=128,
                    maximum=1024,
                    value=512,
                    step=64,
                    label="Max Tokens",
                )
            
            # Submit button
            submit_btn = gr.Button("💊 Get Prescription", variant="primary", size="lg")
        
        with gr.Column(scale=3):
            # Output areas
            prescription_output = gr.Markdown(label="📋 Prescription")
            
            with gr.Accordion("🔍 Raw JSON Output", open=False):
                json_output = gr.Code(label="JSON Response", language="json")
    
    # Example cases
    gr.Examples(
        examples=[
            ["I have severe headache and light sensitivity.", 0.3, 512],
            ["I have nausea and vomiting.", 0.3, 512],
            ["I have joint pain in my knees.", 0.3, 512],
            ["I have persistent cough with phlegm.", 0.3, 512],
            ["I have leg pains and my nails are coming off.", 0.3, 512],
            ["I have difficulty sleeping and anxiety.", 0.3, 512],
        ],
        inputs=[symptoms_input, temperature_slider, max_tokens_slider],
        label="📝 Example Symptoms"
    )
    
    # Connect the interface
    submit_btn.click(
        fn=chat_interface,
        inputs=[symptoms_input, temperature_slider, max_tokens_slider],
        outputs=[prescription_output, json_output]
    )
    
    # Also allow Enter key to submit
    symptoms_input.submit(
        fn=chat_interface,
        inputs=[symptoms_input, temperature_slider, max_tokens_slider],
        outputs=[prescription_output, json_output]
    )

print("✅ Gradio interface ready!")
print("Run: demo.launch() to start the chat interface")


## 🚀 Launch the Chat Interface

Run this cell to start the interactive Gradio interface.


In [ ]:
# Launch the Gradio interface
# Use share=True to get a public link for 72 hours
# Use share=False to run locally only

demo.launch(
    share=False,  # Set to True for public link
    server_name="0.0.0.0",  # Allow external access
    server_port=7860,  # Default Gradio port
    show_error=True,
)
